In [12]:
pip install replicate

DEPRECATION: Loading egg at /Users/jielanzheng/anaconda3/lib/python3.11/site-packages/SQLAlchemy-1.4.23-py3.11-macosx-11.1-arm64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
Note: you may need to restart the kernel to use updated packages.


In [13]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

········


In [3]:
## Load Data
import json

## NewsNLI
with open("QNLI/NewsNLI.json", 'r') as file:
    NewsNLI = json.loads(file.read())
    
## AWPNLI
with open("QNLI/AWPNLI.json", 'r') as file:
    AWPNLI = json.loads(file.read())
    
## RedditNLI
with open("QNLI/RedditNLI.json", 'r') as file:
    RedditNLI = json.loads(file.read())

## RTEQuant
with open("QNLI/RTE_Quant.json", 'r') as file:
    RTEQuant = json.loads(file.read())

## Stress Testing
with open("QNLI/QNLI-Stress Test/QNLI-Stress Test_train.json", 'r') as file:
    QNLIStress = json.loads(file.read())

162


In [4]:
import replicate
from tqdm import tqdm

def qnli_batch(dataset, model_name, result_name):
    answers = {}
    example_prompt = f"""
        Given statement 1 and statement 2, choose between 2 options. Answer one word.
        Example 1: 
        Statement1: {dataset[0]['statement1']}
        Statement2: {dataset[0]['statement2']}
        Options: {dataset[0]['options']}
        Answer: {dataset[0]['answer']}
        Example 2:
        Statement1: {dataset[1]['statement1']}
        Statement2: {dataset[1]['statement2']}
        Options: {dataset[1]['options']}
        Answer: {dataset[1]['answer']}
        Based on the example, answer the following question:
        """
    for i in tqdm(range(len(dataset)-2), desc="Processing Questions"):
        question_prompt = f"""
            Statement1: {dataset[i+2]['statement1']}
            Statement2: {dataset[i+2]['statement2']}
            Options: {dataset[i+2]['options']}
            """
        ans = ''
        for event in replicate.stream(model_name, input={"prompt": example_prompt + question_prompt},):
            ans += str(event)
        answers[i+2] = ans
    
    with open(f'{result_name}.json', 'w', encoding='utf-8') as f:
        json.dump(answers, f, ensure_ascii=False, indent=4)

In [5]:
model_name1 = "meta/meta-llama-3-70b-instruct"
model_name2 = "meta/llama-2-70b-chat"

In [7]:
qnli_batch(NewsNLI, model_name1, 'NewsNLIllama3')

Processing Questions: 100%|███████████████████| 966/966 [06:02<00:00,  2.66it/s]


In [8]:
qnli_batch(NewsNLI, model_name2, 'NewsNLIllama2')

Processing Questions: 100%|███████████████████| 966/966 [35:41<00:00,  2.22s/it]


In [9]:
qnli_batch(AWPNLI, model_name1, 'AWPNLIllama3')

Processing Questions: 100%|███████████████████| 720/720 [04:46<00:00,  2.51it/s]


In [11]:
qnli_batch(AWPNLI, model_name2, 'AWPNLIllama2')

Processing Questions: 100%|███████████████████| 720/720 [27:04<00:00,  2.26s/it]


In [12]:
qnli_batch(RedditNLI, model_name1, 'RedditNLIllama3')

Processing Questions: 100%|███████████████████| 248/248 [01:54<00:00,  2.16it/s]


In [13]:
qnli_batch(RedditNLI, model_name2, 'RedditNLIllama2')

Processing Questions: 100%|███████████████████| 248/248 [09:20<00:00,  2.26s/it]


In [14]:
qnli_batch(RTEQuant, model_name1, 'RTEQuantllama3')

Processing Questions: 100%|███████████████████| 164/164 [00:54<00:00,  3.01it/s]


In [15]:
qnli_batch(RTEQuant, model_name2, 'RTEQuantllama2')

Processing Questions: 100%|███████████████████| 164/164 [05:36<00:00,  2.05s/it]


In [19]:
def qnli_batch_sci(dataset, model_name, result_name):
    answers = {}
    example_prompt = f"""
        Given statement 1 and statement 2, choose between 2 options. Answer one word.
        Example 1: 
        Statement1: {dataset[0]['statement1_sci_10E']}
        Statement2: {dataset[0]['statement2_sci_10E']}
        Options: {dataset[0]['options']}
        Answer: {dataset[0]['answer']}
        Example 2:
        Statement1: {dataset[1]['statement1_sci_10E']}
        Statement2: {dataset[1]['statement2_sci_10E']}
        Options: {dataset[1]['options']}
        Answer: {dataset[1]['answer']}
        Based on the example, answer the following question:
        """
    for i in tqdm(range(len(dataset)-2), desc="Processing Questions"):
        question_prompt = f"""
            Statement1: {dataset[i+2]['statement1_sci_10E']}
            Statement2: {dataset[i+2]['statement2_sci_10E']}
            Options: {dataset[i+2]['options']}
            """
        ans = ''
        for event in replicate.stream(model_name, input={"prompt": example_prompt + question_prompt},):
            ans += str(event)
        answers[i+2] = ans
    
    with open(f'{result_name}_sci.json', 'w', encoding='utf-8') as f:
        json.dump(answers, f, ensure_ascii=False, indent=4)

In [20]:
qnli_batch_sci(NewsNLI, model_name1, 'NewsNLIllama3')

Processing Questions: 100%|███████████████████| 966/966 [07:22<00:00,  2.18it/s]


In [16]:
qnli_batch_sci(AWPNLI, model_name1, 'AWPNLIllama3')

Processing Questions: 100%|███████████████████| 720/720 [04:34<00:00,  2.63it/s]


In [18]:
qnli_batch_sci(RedditNLI, model_name1, 'RedditNLIllama3')

Processing Questions: 100%|███████████████████| 248/248 [01:35<00:00,  2.60it/s]


In [19]:
qnli_batch_sci(RTEQuant, model_name1, 'RTEQuantllama3')

Processing Questions: 100%|███████████████████| 164/164 [00:55<00:00,  2.95it/s]


In [23]:
def qnli_batch_masked(dataset, model_name, result_name):
    answers = {}
    example_prompt = f"""
        Given statement 1 and statement 2, choose between 2 options. Answer one word.
        Example 1: 
        Statement1: {dataset[0]['statement1_mask']}
        Statement2: {dataset[0]['statement2_mask']}
        Options: {dataset[0]['options']}
        Answer: {dataset[0]['answer']}
        Example 2:
        Statement1: {dataset[1]['statement1_mask']}
        Statement2: {dataset[1]['statement2_mask']}
        Options: {dataset[1]['options']}
        Answer: {dataset[1]['answer']}
        Based on the example, answer the following question:
        """
    for i in tqdm(range(len(dataset)-2), desc="Processing Questions"):
        question_prompt = f"""
            Statement1: {dataset[i+2]['statement1_mask']}
            Statement2: {dataset[i+2]['statement2_mask']}
            Options: {dataset[i+2]['options']}
            """
        ans = ''
        for event in replicate.stream(model_name, input={"prompt": example_prompt + question_prompt},):
            ans += str(event)
        answers[i+2] = ans
    
    with open(f'{result_name}_masked.json', 'w', encoding='utf-8') as f:
        json.dump(answers, f, ensure_ascii=False, indent=4)

In [24]:
qnli_batch_masked(NewsNLI, model_name1, 'NewsNLIllama3')

Processing Questions: 100%|███████████████████| 966/966 [07:04<00:00,  2.28it/s]


In [25]:
qnli_batch_masked(AWPNLI, model_name1, 'AWPNLIllama3')

Processing Questions: 100%|███████████████████| 720/720 [05:09<00:00,  2.33it/s]


In [26]:
qnli_batch_masked(RedditNLI, model_name1, 'RedditNLIllama3')

Processing Questions: 100%|███████████████████| 248/248 [02:27<00:00,  1.69it/s]


In [27]:
qnli_batch_masked(RTEQuant, model_name1, 'RTEQuantllama3')

Processing Questions: 100%|███████████████████| 164/164 [01:12<00:00,  2.25it/s]


In [69]:
def evaluation_qnli(results, dataset):
    
    all_labels = set(item['answer'].lower().strip() for item in dataset)
    confusion_matrix = {label: {'tp': 0, 'fp': 0, 'fn': 0} for label in all_labels}
    
    # Populate the confusion matrix
    for i in range(len(dataset)-2):
        
        true_label = dataset[i+2]['answer'].lower().strip()
        predicted_label = results[f'{i+2}'].strip().lower()
        
        for label in all_labels:
            if true_label == label:
                if label in predicted_label:
                    confusion_matrix[label]['tp'] += 1
                else:
                    confusion_matrix[label]['fn'] += 1
            else:
                if label in predicted_label:
                    confusion_matrix[label]['fp'] += 1

    # Calculate precision, recall, and F1 score for each class
    scores = {}
    for label, counts in confusion_matrix.items():
        precision = counts['tp'] / (counts['tp'] + counts['fp']) if (counts['tp'] + counts['fp']) > 0 else 0
        recall = counts['tp'] / (counts['tp'] + counts['fn']) if (counts['tp'] + counts['fn']) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        scores[label] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

    # Calculate macro-average F1 score
    macro_f1_score = sum(score['f1_score'] for score in scores.values()) / len(scores)

    return scores, macro_f1_score

In [75]:
## NewsNLI
with open("NewsNLIllama3.json", 'r') as file:
    NewsNLIllama3 = json.loads(file.read())
with open("NewsNLIllama2.json", 'r') as file:
    NewsNLIllama2 = json.loads(file.read())
    
with open("NewsNLIllama3_sci.json", 'r') as file:
    NewsNLIllama3_sci = json.loads(file.read())
with open("NewsNLIllama3_masked.json", 'r') as file:
    NewsNLIllama3_masked = json.loads(file.read())

print(evaluation_qnli(NewsNLIllama2, NewsNLI))
print(evaluation_qnli(NewsNLIllama3, NewsNLI))
print(evaluation_qnli(NewsNLIllama3_sci, NewsNLI))
print(evaluation_qnli(NewsNLIllama3_masked, NewsNLI))

({'neutral': {'precision': 0.4971687429218573, 'recall': 0.9242105263157895, 'f1_score': 0.646539027982327}, 'entailment': {'precision': 0.6039603960396039, 'recall': 0.2484725050916497, 'f1_score': 0.35209235209235207}}, 0.49931569003733955)
({'neutral': {'precision': 0.8944954128440367, 'recall': 0.4105263157894737, 'f1_score': 0.5627705627705628}, 'entailment': {'precision': 0.6256684491978609, 'recall': 0.9531568228105907, 'f1_score': 0.7554479418886199}}, 0.6591092523295914)
({'neutral': {'precision': 0.8983957219251337, 'recall': 0.35368421052631577, 'f1_score': 0.5075528700906344}, 'entailment': {'precision': 0.6059050064184852, 'recall': 0.9613034623217923, 'f1_score': 0.7433070866141732}}, 0.6254299783524038)
({'neutral': {'precision': 0.8720379146919431, 'recall': 0.3873684210526316, 'f1_score': 0.5364431486880467}, 'entailment': {'precision': 0.614569536423841, 'recall': 0.945010183299389, 'f1_score': 0.7447833065810593}}, 0.640613227634553)


In [76]:
## AWPNLI
with open("AWPNLIllama2.json", 'r') as file:
    AWPNLIllama2 = json.loads(file.read())
with open("AWPNLIllama3.json", 'r') as file:
    AWPNLIllama3 = json.loads(file.read())
    
with open("AWPNLIllama3_sci.json", 'r') as file:
    AWPNLIllama3_sci = json.loads(file.read())
with open("AWPNLIllama3_masked.json", 'r') as file:
    AWPNLIllama3_masked = json.loads(file.read())

print(evaluation_qnli(AWPNLIllama2, AWPNLI))
print(evaluation_qnli(AWPNLIllama3, AWPNLI))
print(evaluation_qnli(AWPNLIllama3_sci, AWPNLI))
print(evaluation_qnli(AWPNLIllama3_masked, AWPNLI))

({'entailment': {'precision': 0.9772727272727273, 'recall': 0.23822714681440443, 'f1_score': 0.3830734966592428}, 'contradiction': {'precision': 0.5013966480446927, 'recall': 1.0, 'f1_score': 0.6679069767441861}}, 0.5254902367017145)
({'entailment': {'precision': 0.5336538461538461, 'recall': 0.9224376731301939, 'f1_score': 0.6761421319796954}, 'contradiction': {'precision': 0.7083333333333334, 'recall': 0.1894150417827298, 'f1_score': 0.29890109890109895}}, 0.4875216154403972)
({'entailment': {'precision': 0.524031007751938, 'recall': 0.9362880886426593, 'f1_score': 0.6719681908548708}, 'contradiction': {'precision': 0.6933333333333334, 'recall': 0.14484679665738162, 'f1_score': 0.23963133640552994}}, 0.45579976363020036)
({'entailment': {'precision': 0.5015432098765432, 'recall': 0.9002770083102493, 'f1_score': 0.6442021803766105}, 'contradiction': {'precision': 0.5, 'recall': 0.10027855153203342, 'f1_score': 0.16705336426914152}}, 0.405627772322876)


In [77]:
## RedditNLI
with open("RedditNLIllama2.json", 'r') as file:
    RedditNLIllama2 = json.loads(file.read())
with open("RedditNLIllama3.json", 'r') as file:
    RedditNLIllama3 = json.loads(file.read())
    
with open("RedditNLIllama3_sci.json", 'r') as file:
    RedditNLIllama3_sci = json.loads(file.read())
with open("RedditNLIllama3_masked.json", 'r') as file:
    RedditNLIllama3_masked = json.loads(file.read())

print(evaluation_qnli(RedditNLIllama2, RedditNLI))
print(evaluation_qnli(RedditNLIllama3, RedditNLI))
print(evaluation_qnli(RedditNLIllama3_sci, RedditNLI))
print(evaluation_qnli(RedditNLIllama3_masked, RedditNLI))

({'neutral': {'precision': 0.625, 'recall': 0.17857142857142858, 'f1_score': 0.2777777777777778}, 'entailment': {'precision': 0.6994219653179191, 'recall': 0.8287671232876712, 'f1_score': 0.7586206896551725}, 'contradiction': {'precision': 0.171875, 'recall': 0.6111111111111112, 'f1_score': 0.2682926829268293}}, 0.4348970501199265)
({'neutral': {'precision': 0.775, 'recall': 0.36904761904761907, 'f1_score': 0.5000000000000001}, 'entailment': {'precision': 0.7287234042553191, 'recall': 0.9383561643835616, 'f1_score': 0.8203592814371258}, 'contradiction': {'precision': 0.4, 'recall': 0.4444444444444444, 'f1_score': 0.4210526315789474}}, 0.5804706376720244)
({'neutral': {'precision': 0.6346153846153846, 'recall': 0.39285714285714285, 'f1_score': 0.4852941176470588}, 'entailment': {'precision': 0.7380952380952381, 'recall': 0.8493150684931506, 'f1_score': 0.7898089171974523}, 'contradiction': {'precision': 0.2857142857142857, 'recall': 0.4444444444444444, 'f1_score': 0.34782608695652173}},

In [78]:
## RTEQuant
with open("RTEQuantllama2.json", 'r') as file:
    RTEQuantllama2 = json.loads(file.read())
with open("RTEQuantllama3.json", 'r') as file:
    RTEQuantllama3 = json.loads(file.read())
    
with open("RTEQuantllama3_sci.json", 'r') as file:
    RTEQuantllama3_sci = json.loads(file.read())
with open("RTEQuantllama3_masked.json", 'r') as file:
    RTEQuantllama3_masked = json.loads(file.read())

print(evaluation_qnli(RTEQuantllama2, RTEQuant))
print(evaluation_qnli(RTEQuantllama3, RTEQuant))
print(evaluation_qnli(RTEQuantllama3_sci, RTEQuant))
print(evaluation_qnli(RTEQuantllama3_masked, RTEQuant))

({'neutral': {'precision': 0.9285714285714286, 'recall': 0.2736842105263158, 'f1_score': 0.4227642276422765}, 'entailment': {'precision': 0.48905109489051096, 'recall': 0.9710144927536232, 'f1_score': 0.6504854368932038}}, 0.5366248322677402)
({'neutral': {'precision': 0.9833333333333333, 'recall': 0.6210526315789474, 'f1_score': 0.7612903225806452}, 'entailment': {'precision': 0.6538461538461539, 'recall': 0.9855072463768116, 'f1_score': 0.7861271676300579}}, 0.7737087451053515)
({'neutral': {'precision': 0.9333333333333333, 'recall': 0.5894736842105263, 'f1_score': 0.7225806451612904}, 'entailment': {'precision': 0.625, 'recall': 0.9420289855072463, 'f1_score': 0.7514450867052023}}, 0.7370128659332463)
({'neutral': {'precision': 1.0, 'recall': 0.49473684210526314, 'f1_score': 0.6619718309859155}, 'entailment': {'precision': 0.5897435897435898, 'recall': 1.0, 'f1_score': 0.7419354838709677}}, 0.7019536574284416)
